$$\psi(x, t) = A \cdot e^{-\frac{(x - x_0)^2}{2\sigma^2}} \cdot e^{i(kx - \omega t)}$$

$$i \hbar \frac{{\partial \psi}}{{\partial t}} = -\frac{\hbar^2}{2m} (\frac{{\partial^2 \psi}}{{\partial x^2}} + \frac{{\partial^2 \psi}}{{\partial y^2}}) + V(x, y) \psi$$

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from scipy.linalg import solve
import matplotlib.pyplot as plt

hbar = 0.6582119514
m = 0.5109989461

# 2D gaussian wave packet
def wavepacket_2d(x, y, x0, y0, sigma_x, sigma_y, kx, ky, A):
    envelope = np.exp(-((x - x0)**2 / (2 * sigma_x**2) + (y - y0)**2 / (2 * sigma_y**2)))
    phase = np.exp(1j * (kx * x + ky * y))
    wave_packet = A * envelope * phase
    return wave_packet

# infinite square well potential
def V_2d(x, y, Lx, Ly):
    V = np.zeros((len(x), len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            if x[i] < -Lx or x[i] > Lx or y[j] < -Ly or y[j] > Ly:
                V[i, j] = float('inf')
    return V

# Crank-Nicholson algorithm for solving the Schrödinger equation
def crank_nicholson(x, y, dx, dy, dt, V, psi0):
    # Number of points
    nx = len(x)
    ny = len(y)

    # Calculate the coefficients
    alpha_x = (1j * dt / (4 * dx**2))*(hbar**2 / (2 * m))
    alpha_y = 1j * dt / (4 * dy**2)*(hbar**2 / (2 * m))
    beta = 1 - 2 * alpha_x - 2 * alpha_y

    # Initialize the wave function
    psi = np.copy(psi0)

    # AX = MY
    for i in range(1, len(V)):
        for j in range(1, len(V)):
            # Construct A
            A = np.diag(1 + 2*alpha_x + 2*alpha_y + 1j*dt*V[i,j]/2)  # diagonal
            A += np.diag(-alpha_x * np.ones(nx - 1), 1)  # upper diagonal
            A += np.diag(-alpha_x * np.ones(nx - 1), -1)  # lower diagonal
            A += np.diag(-alpha_y * np.ones(ny - 1), ny)  # upper diagonal
            A += np.diag(-alpha_y * np.ones(ny - 1), -ny)  # lower diagonal

            # Construct M
            M = np.diag(1 - 2*alpha_x - 2*alpha_y - 1j*dt*(V[i,j])/2)
            M += np.diag(alpha_x * np.ones(nx - 1), 1)
            M += np.diag(alpha_x * np.ones(nx - 1), -1)
            M += np.diag(alpha_y * np.ones(ny - 1), ny)
            M += np.diag(alpha_y * np.ones(ny - 1), -ny)

            # Solve the linear system
            psi = solve(A, np.dot(M, psi))

    #for t in range(1, len(V)):
    print(A)
    return psi

    

In [27]:
# Example usage
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, y)
wave_packet = wavepacket_2d(X, Y, 0, 0, 1, 1, 1, 1, 1)
V = V_2d(x, y, 9, 9)
crank_nicholson(x, y, 0.1, 0.1, 0.1, V, wave_packet)



ValueError: Input must be 1- or 2-d.

In [ ]:
plt.imshow(np.real(wave_packet), extent=[-10, 10, -10, 10])
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')
plt.title('2D Gaussian Wave Packet')
plt.show()

# 3D plot
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, np.abs(wave_packet), cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('2D Gaussian Wave Packet')
plt.show()

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, V, cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Infinite Square Well Potential')
plt.show()